## 징역

In [5]:
import pickle
import re
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.sequence import pad_sequences
from PyKomoran import *
from tensorflow.keras.models import load_model

okt = Okt()
# komoran = Komoran("EXP")
model = load_model("./model/git_imprisonment_231119.keras")

# 불용어 사전 정의
stopwords = set()
with open("../data-preprocessing/user_dic/stopwords", "r") as f:
    for line in f.readlines():
        stopwords.add(line.strip())

def spacing_okt(wrongSentence):
    tagged = okt.pos(wrongSentence)
    corrected = ""
    for i in tagged:
        if i[1] in ('Josa', 'PreEomi', 'Eomi', 'Suffix', 'Punctuation'):
            corrected += i[0]
        else:
            corrected += " "+i[0]
    if corrected[0] == " ":
        corrected = corrected[1:]
    return corrected

def text_tokenize(tokenizer, text):
    # 텍스트 클렌징
    text = text.replace(',','').replace('"','').replace('\'','').replace('.','').replace('(',' ').\
            replace(')','').replace('!','').replace('?','').replace(':','').replace(';','').lower()
    text = text.replace("\n"," ")
    text = re.sub(r'\d+?\.\s\d+\.\s\d+\.', '', text) # 날짜 제거
    text = re.sub(r'\b\w+법원|지원', '', text) # 법원명 제거
    text = re.sub('수사보고|범 죄 사 실|범죄사실', '', text) # 판례의 기본적인 문구 제거
    text = re.sub(r'[「」『』\[\],.:%○]', '', text) # 특수기호 제거

    # 띄어쓰기 교정
    text = spacing_okt(text)

    # 명사 추출
    nouns = []
    for noun in okt.nouns(text):
        if noun.isalpha() and (noun not in stopwords): # 단어에 숫자가 아닌 경우에만 저장 and 불용어 사전에 포함되지 않는 경우
            nouns.append(noun)

    out = tokenizer.texts_to_sequences([nouns])
    return pad_sequences(out, 256)


tokenizer = {}
with open('./model/tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

print(model.predict(text_tokenize(tokenizer, "초등학교 앞 사거리에서 음주운전")))

print(model.predict(text_tokenize(tokenizer, "초등학교 앞 사거리에서 음주운전 및 과속")))

print(model.predict(text_tokenize(tokenizer, "중학교에서 음주운전")))

print(model.predict(text_tokenize(tokenizer, "어린이보호구역에서 음주운전 및 과속")))

print(model.predict(text_tokenize(tokenizer, "어린이 보호 구역에서 음주운전 및 과속")))

print(model.predict(text_tokenize(tokenizer, "어린이 보호구역에서 음주운전 및 과속")))

# 실제 징역 30개월
print(model.predict(text_tokenize(tokenizer, "범 죄 사 실  [범죄전력]  피고인은 2020. 12. 14. 서울동부지방법원에서 도로교통법위반(음주운전)죄 등으로 징역 2년에 집행유예 3년을 선고받고 2020. 12. 22. 그 판결이 확정되어 집행유예 기간 중이다.  『2022고단1587』  1. 특정범죄가중처벌등에관한법률위반(도주치상), 도로교통법위반(사고후미조치), 도로교통법위반  피고인은 (차량번호 1 생략) 카니발 차량의 운전업무에 종사하는 사람이다.  피고인은 2022. 6. 6. 04:25경 서울 광진구 천호대로 502 군자교 사거리 앞 편도 5차로 도로를 군자교 방면에서 군자역 방면으로 5차로를 따라 진행하게 되었다. 당시 그곳은 신호등이 있는 교차로로서 이러한 경우 자동차의 운전업무에 종사하는 사람에게는 술에 취하지 않은 상태로 신호를 준수하면서 운전하여 사고를 미연에 방지하여야 할 업무상 주의의무가 있었다.  그럼에도 불구하고 피고인은 이를 게을리 한 채 술에 취한 상태에서 전방 적색신호 등에 신호를 무시하고 그대로 진행한 과실로 피고인 차량 진행 방향 우측에서 좌측으로 신호에 따라 진행하는 피해자 B(42세)이 운전하는 (차량번호 2 생략) 그랜저 승용차의 좌측면 부분을 피고인 차량 전면부 부분으로 충돌하였다.  피고인은 위와 같은 업무상 과실로 피해자에게 약 28일간의 치료가 필요한 늑골 골절, 좌측 5번, 6번의 상해를 입게 함과 동시에 위 그랜저 차량에 대해 수리비가 9,374,000원이 들도록 손괴하고도 곧 정차하여 피해자를 구호하는 등 필요한 조치를 취하지 아니하고 그대로 도주하였다.  2. 도로교통법위반(음주운전), 도로교통법위반(무면허운전)  피고인은 자동차운전면허를 받지 아니하고 혈중알코올농도 0.116%의 술에 취한 상태로 서울 용산구 이태원동 불상의 도로에서부터 서울 광진구 C 앞 도로에 이르기까지 약 10km 구간에서 위 (차량번호 1 생략) 카니발 차량을 운전하였다.  『2022고단1865』  피고인은 2021. 12. 18. 04:55경 서울 동대문구 D에서 같은 구 E 앞 도로에 이르기까지 자동차운전면허를 받지 아니하고 혈중알코올농도 0.102%의 술에 취한 상태로 (차량번호 1 생략) 카니발 승용차를 운전하였다.  증거의 요지  『2022고단1587』  1. 피고인의 법정진술  1. 교통사고발생상황진술서  1. 실황조사서, 현장 경찰관 메모지, 112신고사건처리표, (차량번호 1 생략) 앞 번호판사진, 차적조회, 자동차운전면허대장, 의무보험조회, 주취운전자정황진술보고서, 음주운전단속결과조회, 음주출력지, 상해진단서, 자동차 점검, 정비 명세서, 사고영상 캡처 사진, (차량번호 1 생략) 카니발 블랙박스 영상 캡처 사진, 보험가입증명서, 문자 수,발신 내용 캡처 사진, 보상처리확인서(개인보험 청구용)  1. 압수조서 및 압수목록  1. 각 수사보고서  『2022고단1865』  1. 피고인의 법정진술  1. 주취운전정황보고서, 음주측정기록지, 단속경위서, 운전면허대장, 본청결격조회, 차적조회, 112신고사건처리 내역서 송부  1. 각 수사보고서  [판시 전과]  1. 범죄경력등조회회보서  1. 수사보고서(피의자에 대한 동종전력 판결서 등 첨부)  법령의 적용  1. 범죄사실에 대한 해당법조  특정범죄 가중처벌 등에 관한 법률 제5조의3 제1항 제2호, 형법 제268조(업무상과 실치상 후 도주의 점), 도로교통법 제148조, 제54조 제1항(물건손괴 후 미조치의 점), 도로교통법 제151조(과실재물손괴의 점), 각 도로교통법 제148조의2 제3항 제2호, 제44조 제1항(음주운전의 점), 각 구 도로교통법(2021. 10. 19. 법률 제18491호로 개정되어 2022. 10. 20. 시행되기 전의 것) 제152조 제1호, 제43조(무면허운전의 점)  1. 상상적 경합  형법 제40조, 제50조  1. 형의 선택  도로교통법위반죄에 대하여는 금고형을 선택, 나머지 각 죄에 대하여는 징역형 선택  1. 경합범가중  형법 제37조 전단, 제38조 제1항 제2호, 제2항, 제50조  1. 배상신청 각하  소송촉진 등에 관한 특례법 제32조 제1항 제3호, 제25조 제3항 제3호(피고인의 배상책임 범위가 명백하다고 보기 어려움)  양형의 이유  피고인은 2010년에 특정범죄가중처벌등에관한법률위반(위험운전치사상)죄 및 도로교통법위반(음주운전)죄로, 2016년에 도로교통법위반(음주운전)죄로, 2020년에 도로교통법위반(음주운전)죄, 도로교통법위반(무면허운전)죄로 각 벌금형으로 처벌받았다. 또한 피고인은 2020. 12. 14. 서울동부지방법원에서 도로교통법위반(음주운전)죄 등으로 징역 2년에 집행유예 3년을 선고받고 2020. 12. 22. 그 판결이 확정되어 집행유예 기간 중임에도, 2021. 12. 18.에는 도로교통법위반(무면허운전) 및 도로교통법위반(음주운전) 각 범행을 저질렀고(2022고단1865 사건 범행), 2022. 6. 6.에는 음주 및 무면허 상태에서 운전을 하다가 교통사고를 일으켜 인적, 물적 피해를 발생시켰음에도 그대로 도주하기도 하였다(2022고단1587 사건 범행). 나아가 2022고단1587 사건의 피해자가 피고인의 처벌을 원하고, 위 피해자의 피해가 모두 회복되지 못한 점 등을 이울러 고려하면, 피고인에 대해서는 엄벌을 하여야만 한다.  다만, 피고인이 자신의 잘못을 인정하고 있는 점, 피고인의 차량이 F공제조합에 가입되어 있어 위 피해자의 치료비가 보험처리 된 점, 그 밖에 피고인의 연령, 성행과 환경, 범행의 동기, 수단과 결과, 범행 후의 정황 등 기록 및 변론에 나타난 여러 양형조건을 참작하여 주문과 같이 형을 결정한다.")))

#실제 징역 6개월, 집행유예 2년
print(model.predict(text_tokenize(tokenizer, "범 죄 사 실  1. 도로교통법위반(음주운전), 도로교통법위반(무면허운전)  피고인은 2022. 9. 20. 16:44경 서울 송파구 B에 있는 C역 부근 앞 도로에서부터 같은 구 D 앞 도로에 이르기까지 약 100m 구간에서 원동기장치자전거 운전면허 없이 혈중알코올농도 0.06%의 술에 취한 상태로 번호판이 없는 50cc 오토바이를 운전하였다.  2. 자동차손해배상보장법위반  누구든지 의무보험에 가입되어 있지 아니한 자동차를 도로에서 운행하여서는 아니 된다.  피고인은 제1항 기재 일시, 장소에서 의무보험에 가입되지 아니한 위 오토바이를 운행하였다.  증거의 요지  1. 피고인의 법정진술  1. 음주운전단속결과통보, 주취운전자정황진술보고서  1. 자동차운전면허대장  1. 사진  법령의 적용  1. 범죄사실에 대한 해당법조  도로교통법 제148조의2 제3항 제2호, 제44조 제1항(음주운전의 점, 징역형 선택), 도로교통법 제154조 제2호, 제43조(원동기장치자전거 무면허운전의 점), 자동차손해배상 보장법 제46조 제3항 제2호, 제8조 본문(의무보험 미가입 차량 운행의 점)  1. 상상적 경합  형법 제40조, 제50조  1. 형의 선택  징역형 선택  1. 경합범가중  형법 제37조 전단, 제38조 제1항 제2호, 제50조  1. 정상참작감경  형법 제53조, 제55조 제1항 제3호  1. 집행유예  형법 제62조 제1항  1. 수강명령  형법 제62조의2  양형의 이유  피고인이 음주운전으로 3회, 무면허운전으로 1회 처벌받은 전력이 있음에도 이 사건 각 범행을 저질렀다. 피고인의 혈중알코올농도, 운전 거리, 적발 경위, 그 밖에 피고인의 연령, 성행, 환경, 범행의 동기, 수단과 결과, 범행 후의 정황 등 이 사건 기록 및 변론에 나타난 여러 양형 조건들을 종합하여 주문과 같이 형을 정한다.")))

1/1 [==============================] - 0s 100ms/step
[[3.8373156]]
1/1 [==============================] - 0s 24ms/step
[[4.3817887]]
1/1 [==============================] - 0s 21ms/step
[[2.4269195]]
1/1 [==============================] - 0s 22ms/step
[[3.30307]]
1/1 [==============================] - 0s 25ms/step
[[3.30307]]
1/1 [==============================] - 0s 22ms/step
[[3.30307]]
1/1 [==============================] - 0s 19ms/step
[[13.887294]]
1/1 [==============================] - 0s 19ms/step
[[8.513321]]


## 집행유예

In [11]:
import pickle
import re
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model

okt = Okt()
model = load_model("./model/git_probation_231120.keras")

# 불용어 사전 정의
stopwords = set()
with open("../data-preprocessing/user_dic/stopwords", "r") as f:
    for line in f.readlines():
        stopwords.add(line.strip())

def spacing_okt(wrongSentence):
    tagged = okt.pos(wrongSentence)
    corrected = ""
    for i in tagged:
        if i[1] in ('Josa', 'PreEomi', 'Eomi', 'Suffix', 'Punctuation'):
            corrected += i[0]
        else:
            corrected += " "+i[0]
    if corrected[0] == " ":
        corrected = corrected[1:]
    return corrected

def text_tokenize(tokenizer, text):
    # 텍스트 클렌징
    text = text.replace(',','').replace('"','').replace('\'','').replace('.','').replace('(',' ').\
            replace(')','').replace('!','').replace('?','').replace(':','').replace(';','').lower()
    text = text.replace("\n"," ")
    text = re.sub(r'\d+?\.\s\d+\.\s\d+\.', '', text) # 날짜 제거
    text = re.sub(r'\b\w+법원|지원', '', text) # 법원명 제거
    text = re.sub('수사보고|범 죄 사 실|범죄사실', '', text) # 판례의 기본적인 문구 제거
    text = re.sub(r'[「」『』\[\],.:%○]', '', text) # 특수기호 제거

    # 띄어쓰기 교정
    text = spacing_okt(text)

    # 명사 추출
    nouns = []
    for noun in okt.nouns(text):
        if noun.isalpha() and (noun not in stopwords): # 단어에 숫자가 아닌 경우에만 저장 and 불용어 사전에 포함되지 않는 경우
            nouns.append(noun)

    out = tokenizer.texts_to_sequences([nouns])
    return pad_sequences(out, 256)


tokenizer = {}
with open('./model/tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

print(model.predict(text_tokenize(tokenizer, "초등학교 앞 사거리에서 음주운전")))

print(model.predict(text_tokenize(tokenizer, "초등학교 앞 사거리에서 음주운전 및 과속")))

print(model.predict(text_tokenize(tokenizer, "중학교에서 음주운전")))

print(model.predict(text_tokenize(tokenizer, "어린이보호구역에서 음주운전 및 과속")))

print(model.predict(text_tokenize(tokenizer, "어린이 보호 구역에서 음주운전 및 과속")))

print(model.predict(text_tokenize(tokenizer, "어린이 보호구역에서 음주운전 및 과속")))

# 실제 징역 30개월
print(model.predict(text_tokenize(tokenizer, "범 죄 사 실  [범죄전력]  피고인은 2020. 12. 14. 서울동부지방법원에서 도로교통법위반(음주운전)죄 등으로 징역 2년에 집행유예 3년을 선고받고 2020. 12. 22. 그 판결이 확정되어 집행유예 기간 중이다.  『2022고단1587』  1. 특정범죄가중처벌등에관한법률위반(도주치상), 도로교통법위반(사고후미조치), 도로교통법위반  피고인은 (차량번호 1 생략) 카니발 차량의 운전업무에 종사하는 사람이다.  피고인은 2022. 6. 6. 04:25경 서울 광진구 천호대로 502 군자교 사거리 앞 편도 5차로 도로를 군자교 방면에서 군자역 방면으로 5차로를 따라 진행하게 되었다. 당시 그곳은 신호등이 있는 교차로로서 이러한 경우 자동차의 운전업무에 종사하는 사람에게는 술에 취하지 않은 상태로 신호를 준수하면서 운전하여 사고를 미연에 방지하여야 할 업무상 주의의무가 있었다.  그럼에도 불구하고 피고인은 이를 게을리 한 채 술에 취한 상태에서 전방 적색신호 등에 신호를 무시하고 그대로 진행한 과실로 피고인 차량 진행 방향 우측에서 좌측으로 신호에 따라 진행하는 피해자 B(42세)이 운전하는 (차량번호 2 생략) 그랜저 승용차의 좌측면 부분을 피고인 차량 전면부 부분으로 충돌하였다.  피고인은 위와 같은 업무상 과실로 피해자에게 약 28일간의 치료가 필요한 늑골 골절, 좌측 5번, 6번의 상해를 입게 함과 동시에 위 그랜저 차량에 대해 수리비가 9,374,000원이 들도록 손괴하고도 곧 정차하여 피해자를 구호하는 등 필요한 조치를 취하지 아니하고 그대로 도주하였다.  2. 도로교통법위반(음주운전), 도로교통법위반(무면허운전)  피고인은 자동차운전면허를 받지 아니하고 혈중알코올농도 0.116%의 술에 취한 상태로 서울 용산구 이태원동 불상의 도로에서부터 서울 광진구 C 앞 도로에 이르기까지 약 10km 구간에서 위 (차량번호 1 생략) 카니발 차량을 운전하였다.  『2022고단1865』  피고인은 2021. 12. 18. 04:55경 서울 동대문구 D에서 같은 구 E 앞 도로에 이르기까지 자동차운전면허를 받지 아니하고 혈중알코올농도 0.102%의 술에 취한 상태로 (차량번호 1 생략) 카니발 승용차를 운전하였다.  증거의 요지  『2022고단1587』  1. 피고인의 법정진술  1. 교통사고발생상황진술서  1. 실황조사서, 현장 경찰관 메모지, 112신고사건처리표, (차량번호 1 생략) 앞 번호판사진, 차적조회, 자동차운전면허대장, 의무보험조회, 주취운전자정황진술보고서, 음주운전단속결과조회, 음주출력지, 상해진단서, 자동차 점검, 정비 명세서, 사고영상 캡처 사진, (차량번호 1 생략) 카니발 블랙박스 영상 캡처 사진, 보험가입증명서, 문자 수,발신 내용 캡처 사진, 보상처리확인서(개인보험 청구용)  1. 압수조서 및 압수목록  1. 각 수사보고서  『2022고단1865』  1. 피고인의 법정진술  1. 주취운전정황보고서, 음주측정기록지, 단속경위서, 운전면허대장, 본청결격조회, 차적조회, 112신고사건처리 내역서 송부  1. 각 수사보고서  [판시 전과]  1. 범죄경력등조회회보서  1. 수사보고서(피의자에 대한 동종전력 판결서 등 첨부)  법령의 적용  1. 범죄사실에 대한 해당법조  특정범죄 가중처벌 등에 관한 법률 제5조의3 제1항 제2호, 형법 제268조(업무상과 실치상 후 도주의 점), 도로교통법 제148조, 제54조 제1항(물건손괴 후 미조치의 점), 도로교통법 제151조(과실재물손괴의 점), 각 도로교통법 제148조의2 제3항 제2호, 제44조 제1항(음주운전의 점), 각 구 도로교통법(2021. 10. 19. 법률 제18491호로 개정되어 2022. 10. 20. 시행되기 전의 것) 제152조 제1호, 제43조(무면허운전의 점)  1. 상상적 경합  형법 제40조, 제50조  1. 형의 선택  도로교통법위반죄에 대하여는 금고형을 선택, 나머지 각 죄에 대하여는 징역형 선택  1. 경합범가중  형법 제37조 전단, 제38조 제1항 제2호, 제2항, 제50조  1. 배상신청 각하  소송촉진 등에 관한 특례법 제32조 제1항 제3호, 제25조 제3항 제3호(피고인의 배상책임 범위가 명백하다고 보기 어려움)  양형의 이유  피고인은 2010년에 특정범죄가중처벌등에관한법률위반(위험운전치사상)죄 및 도로교통법위반(음주운전)죄로, 2016년에 도로교통법위반(음주운전)죄로, 2020년에 도로교통법위반(음주운전)죄, 도로교통법위반(무면허운전)죄로 각 벌금형으로 처벌받았다. 또한 피고인은 2020. 12. 14. 서울동부지방법원에서 도로교통법위반(음주운전)죄 등으로 징역 2년에 집행유예 3년을 선고받고 2020. 12. 22. 그 판결이 확정되어 집행유예 기간 중임에도, 2021. 12. 18.에는 도로교통법위반(무면허운전) 및 도로교통법위반(음주운전) 각 범행을 저질렀고(2022고단1865 사건 범행), 2022. 6. 6.에는 음주 및 무면허 상태에서 운전을 하다가 교통사고를 일으켜 인적, 물적 피해를 발생시켰음에도 그대로 도주하기도 하였다(2022고단1587 사건 범행). 나아가 2022고단1587 사건의 피해자가 피고인의 처벌을 원하고, 위 피해자의 피해가 모두 회복되지 못한 점 등을 이울러 고려하면, 피고인에 대해서는 엄벌을 하여야만 한다.  다만, 피고인이 자신의 잘못을 인정하고 있는 점, 피고인의 차량이 F공제조합에 가입되어 있어 위 피해자의 치료비가 보험처리 된 점, 그 밖에 피고인의 연령, 성행과 환경, 범행의 동기, 수단과 결과, 범행 후의 정황 등 기록 및 변론에 나타난 여러 양형조건을 참작하여 주문과 같이 형을 결정한다.")))

#실제 징역 6개월, 집행유예 2년
print(model.predict(text_tokenize(tokenizer, "범 죄 사 실  1. 도로교통법위반(음주운전), 도로교통법위반(무면허운전)  피고인은 2022. 9. 20. 16:44경 서울 송파구 B에 있는 C역 부근 앞 도로에서부터 같은 구 D 앞 도로에 이르기까지 약 100m 구간에서 원동기장치자전거 운전면허 없이 혈중알코올농도 0.06%의 술에 취한 상태로 번호판이 없는 50cc 오토바이를 운전하였다.  2. 자동차손해배상보장법위반  누구든지 의무보험에 가입되어 있지 아니한 자동차를 도로에서 운행하여서는 아니 된다.  피고인은 제1항 기재 일시, 장소에서 의무보험에 가입되지 아니한 위 오토바이를 운행하였다.  증거의 요지  1. 피고인의 법정진술  1. 음주운전단속결과통보, 주취운전자정황진술보고서  1. 자동차운전면허대장  1. 사진  법령의 적용  1. 범죄사실에 대한 해당법조  도로교통법 제148조의2 제3항 제2호, 제44조 제1항(음주운전의 점, 징역형 선택), 도로교통법 제154조 제2호, 제43조(원동기장치자전거 무면허운전의 점), 자동차손해배상 보장법 제46조 제3항 제2호, 제8조 본문(의무보험 미가입 차량 운행의 점)  1. 상상적 경합  형법 제40조, 제50조  1. 형의 선택  징역형 선택  1. 경합범가중  형법 제37조 전단, 제38조 제1항 제2호, 제50조  1. 정상참작감경  형법 제53조, 제55조 제1항 제3호  1. 집행유예  형법 제62조 제1항  1. 수강명령  형법 제62조의2  양형의 이유  피고인이 음주운전으로 3회, 무면허운전으로 1회 처벌받은 전력이 있음에도 이 사건 각 범행을 저질렀다. 피고인의 혈중알코올농도, 운전 거리, 적발 경위, 그 밖에 피고인의 연령, 성행, 환경, 범행의 동기, 수단과 결과, 범행 후의 정황 등 이 사건 기록 및 변론에 나타난 여러 양형 조건들을 종합하여 주문과 같이 형을 정한다.")))

1/1 [==============================] - 0s 123ms/step
[[0.27113348]]
1/1 [==============================] - 0s 22ms/step
[[0.2665929]]
1/1 [==============================] - 0s 24ms/step
[[0.31660816]]
1/1 [==============================] - 0s 22ms/step
[[0.30902722]]
1/1 [==============================] - 0s 21ms/step
[[0.30902722]]
1/1 [==============================] - 0s 22ms/step
[[0.30902722]]
1/1 [==============================] - 0s 24ms/step
[[0.13907778]]
1/1 [==============================] - 0s 21ms/step
[[0.9999733]]


## 벌금

In [23]:
import pickle
import re
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model

okt = Okt()
model = load_model("./model/fine_231122.keras")

# 불용어 사전 정의
stopwords = set()
with open("../data-preprocessing/user_dic/stopwords", "r") as f:
    for line in f.readlines():
        stopwords.add(line.strip())

def spacing_okt(wrongSentence):
    tagged = okt.pos(wrongSentence)
    corrected = ""
    for i in tagged:
        if i[1] in ('Josa', 'PreEomi', 'Eomi', 'Suffix', 'Punctuation'):
            corrected += i[0]
        else:
            corrected += " "+i[0]
    if corrected[0] == " ":
        corrected = corrected[1:]
    return corrected

def text_tokenize(tokenizer, text):
    # 텍스트 클렌징
    text = text.replace(',','').replace('"','').replace('\'','').replace('.','').replace('(',' ').\
            replace(')','').replace('!','').replace('?','').replace(':','').replace(';','').lower()
    text = text.replace("\n"," ")
    text = re.sub(r'\d+?\.\s\d+\.\s\d+\.', '', text) # 날짜 제거
    text = re.sub(r'\b\w+법원|지원', '', text) # 법원명 제거
    text = re.sub('수사보고|범 죄 사 실|범죄사실', '', text) # 판례의 기본적인 문구 제거
    text = re.sub(r'[「」『』\[\],.:%○]', '', text) # 특수기호 제거

    # 띄어쓰기 교정
    text = spacing_okt(text)

    # 명사 추출
    nouns = []
    for noun in okt.nouns(text):
        if noun.isalpha() and (noun not in stopwords): # 단어에 숫자가 아닌 경우에만 저장 and 불용어 사전에 포함되지 않는 경우
            nouns.append(noun)

    out = tokenizer.texts_to_sequences([nouns])
    return pad_sequences(out, 256)


tokenizer = {}
with open('./model/tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

print(model.predict(text_tokenize(tokenizer, "초등학교 앞 사거리에서 음주운전")))

print(model.predict(text_tokenize(tokenizer, "초등학교 앞 사거리에서 음주운전 및 과속")))

print(model.predict(text_tokenize(tokenizer, "중학교에서 음주운전")))

print(model.predict(text_tokenize(tokenizer, "어린이보호구역에서 음주운전 및 과속")))

print(model.predict(text_tokenize(tokenizer, "어린이 보호 구역에서 음주운전 및 과속")))

print(model.predict(text_tokenize(tokenizer, "어린이 보호구역에서 음주운전 및 과속")))

# 실제 징역 30개월, 벌금 없음
print(model.predict(text_tokenize(tokenizer, "범 죄 사 실  [범죄전력]  피고인은 2020. 12. 14. 서울동부지방법원에서 도로교통법위반(음주운전)죄 등으로 징역 2년에 집행유예 3년을 선고받고 2020. 12. 22. 그 판결이 확정되어 집행유예 기간 중이다.  『2022고단1587』  1. 특정범죄가중처벌등에관한법률위반(도주치상), 도로교통법위반(사고후미조치), 도로교통법위반  피고인은 (차량번호 1 생략) 카니발 차량의 운전업무에 종사하는 사람이다.  피고인은 2022. 6. 6. 04:25경 서울 광진구 천호대로 502 군자교 사거리 앞 편도 5차로 도로를 군자교 방면에서 군자역 방면으로 5차로를 따라 진행하게 되었다. 당시 그곳은 신호등이 있는 교차로로서 이러한 경우 자동차의 운전업무에 종사하는 사람에게는 술에 취하지 않은 상태로 신호를 준수하면서 운전하여 사고를 미연에 방지하여야 할 업무상 주의의무가 있었다.  그럼에도 불구하고 피고인은 이를 게을리 한 채 술에 취한 상태에서 전방 적색신호 등에 신호를 무시하고 그대로 진행한 과실로 피고인 차량 진행 방향 우측에서 좌측으로 신호에 따라 진행하는 피해자 B(42세)이 운전하는 (차량번호 2 생략) 그랜저 승용차의 좌측면 부분을 피고인 차량 전면부 부분으로 충돌하였다.  피고인은 위와 같은 업무상 과실로 피해자에게 약 28일간의 치료가 필요한 늑골 골절, 좌측 5번, 6번의 상해를 입게 함과 동시에 위 그랜저 차량에 대해 수리비가 9,374,000원이 들도록 손괴하고도 곧 정차하여 피해자를 구호하는 등 필요한 조치를 취하지 아니하고 그대로 도주하였다.  2. 도로교통법위반(음주운전), 도로교통법위반(무면허운전)  피고인은 자동차운전면허를 받지 아니하고 혈중알코올농도 0.116%의 술에 취한 상태로 서울 용산구 이태원동 불상의 도로에서부터 서울 광진구 C 앞 도로에 이르기까지 약 10km 구간에서 위 (차량번호 1 생략) 카니발 차량을 운전하였다.  『2022고단1865』  피고인은 2021. 12. 18. 04:55경 서울 동대문구 D에서 같은 구 E 앞 도로에 이르기까지 자동차운전면허를 받지 아니하고 혈중알코올농도 0.102%의 술에 취한 상태로 (차량번호 1 생략) 카니발 승용차를 운전하였다.  증거의 요지  『2022고단1587』  1. 피고인의 법정진술  1. 교통사고발생상황진술서  1. 실황조사서, 현장 경찰관 메모지, 112신고사건처리표, (차량번호 1 생략) 앞 번호판사진, 차적조회, 자동차운전면허대장, 의무보험조회, 주취운전자정황진술보고서, 음주운전단속결과조회, 음주출력지, 상해진단서, 자동차 점검, 정비 명세서, 사고영상 캡처 사진, (차량번호 1 생략) 카니발 블랙박스 영상 캡처 사진, 보험가입증명서, 문자 수,발신 내용 캡처 사진, 보상처리확인서(개인보험 청구용)  1. 압수조서 및 압수목록  1. 각 수사보고서  『2022고단1865』  1. 피고인의 법정진술  1. 주취운전정황보고서, 음주측정기록지, 단속경위서, 운전면허대장, 본청결격조회, 차적조회, 112신고사건처리 내역서 송부  1. 각 수사보고서  [판시 전과]  1. 범죄경력등조회회보서  1. 수사보고서(피의자에 대한 동종전력 판결서 등 첨부)  법령의 적용  1. 범죄사실에 대한 해당법조  특정범죄 가중처벌 등에 관한 법률 제5조의3 제1항 제2호, 형법 제268조(업무상과 실치상 후 도주의 점), 도로교통법 제148조, 제54조 제1항(물건손괴 후 미조치의 점), 도로교통법 제151조(과실재물손괴의 점), 각 도로교통법 제148조의2 제3항 제2호, 제44조 제1항(음주운전의 점), 각 구 도로교통법(2021. 10. 19. 법률 제18491호로 개정되어 2022. 10. 20. 시행되기 전의 것) 제152조 제1호, 제43조(무면허운전의 점)  1. 상상적 경합  형법 제40조, 제50조  1. 형의 선택  도로교통법위반죄에 대하여는 금고형을 선택, 나머지 각 죄에 대하여는 징역형 선택  1. 경합범가중  형법 제37조 전단, 제38조 제1항 제2호, 제2항, 제50조  1. 배상신청 각하  소송촉진 등에 관한 특례법 제32조 제1항 제3호, 제25조 제3항 제3호(피고인의 배상책임 범위가 명백하다고 보기 어려움)  양형의 이유  피고인은 2010년에 특정범죄가중처벌등에관한법률위반(위험운전치사상)죄 및 도로교통법위반(음주운전)죄로, 2016년에 도로교통법위반(음주운전)죄로, 2020년에 도로교통법위반(음주운전)죄, 도로교통법위반(무면허운전)죄로 각 벌금형으로 처벌받았다. 또한 피고인은 2020. 12. 14. 서울동부지방법원에서 도로교통법위반(음주운전)죄 등으로 징역 2년에 집행유예 3년을 선고받고 2020. 12. 22. 그 판결이 확정되어 집행유예 기간 중임에도, 2021. 12. 18.에는 도로교통법위반(무면허운전) 및 도로교통법위반(음주운전) 각 범행을 저질렀고(2022고단1865 사건 범행), 2022. 6. 6.에는 음주 및 무면허 상태에서 운전을 하다가 교통사고를 일으켜 인적, 물적 피해를 발생시켰음에도 그대로 도주하기도 하였다(2022고단1587 사건 범행). 나아가 2022고단1587 사건의 피해자가 피고인의 처벌을 원하고, 위 피해자의 피해가 모두 회복되지 못한 점 등을 이울러 고려하면, 피고인에 대해서는 엄벌을 하여야만 한다.  다만, 피고인이 자신의 잘못을 인정하고 있는 점, 피고인의 차량이 F공제조합에 가입되어 있어 위 피해자의 치료비가 보험처리 된 점, 그 밖에 피고인의 연령, 성행과 환경, 범행의 동기, 수단과 결과, 범행 후의 정황 등 기록 및 변론에 나타난 여러 양형조건을 참작하여 주문과 같이 형을 결정한다.")))

#실제 징역 6개월, 집행유예 2년, 벌금 없음
print(model.predict(text_tokenize(tokenizer, "범 죄 사 실  1. 도로교통법위반(음주운전), 도로교통법위반(무면허운전)  피고인은 2022. 9. 20. 16:44경 서울 송파구 B에 있는 C역 부근 앞 도로에서부터 같은 구 D 앞 도로에 이르기까지 약 100m 구간에서 원동기장치자전거 운전면허 없이 혈중알코올농도 0.06%의 술에 취한 상태로 번호판이 없는 50cc 오토바이를 운전하였다.  2. 자동차손해배상보장법위반  누구든지 의무보험에 가입되어 있지 아니한 자동차를 도로에서 운행하여서는 아니 된다.  피고인은 제1항 기재 일시, 장소에서 의무보험에 가입되지 아니한 위 오토바이를 운행하였다.  증거의 요지  1. 피고인의 법정진술  1. 음주운전단속결과통보, 주취운전자정황진술보고서  1. 자동차운전면허대장  1. 사진  법령의 적용  1. 범죄사실에 대한 해당법조  도로교통법 제148조의2 제3항 제2호, 제44조 제1항(음주운전의 점, 징역형 선택), 도로교통법 제154조 제2호, 제43조(원동기장치자전거 무면허운전의 점), 자동차손해배상 보장법 제46조 제3항 제2호, 제8조 본문(의무보험 미가입 차량 운행의 점)  1. 상상적 경합  형법 제40조, 제50조  1. 형의 선택  징역형 선택  1. 경합범가중  형법 제37조 전단, 제38조 제1항 제2호, 제50조  1. 정상참작감경  형법 제53조, 제55조 제1항 제3호  1. 집행유예  형법 제62조 제1항  1. 수강명령  형법 제62조의2  양형의 이유  피고인이 음주운전으로 3회, 무면허운전으로 1회 처벌받은 전력이 있음에도 이 사건 각 범행을 저질렀다. 피고인의 혈중알코올농도, 운전 거리, 적발 경위, 그 밖에 피고인의 연령, 성행, 환경, 범행의 동기, 수단과 결과, 범행 후의 정황 등 이 사건 기록 및 변론에 나타난 여러 양형 조건들을 종합하여 주문과 같이 형을 정한다.")))

# 벌금 1,300만원
print(model.predict(text_tokenize(tokenizer, "범죄사실  피고인은 2012. 5. 4. 수원지방법원 안양지원에서 도로교통법위반(음주운전)죄 등으로 벌금 250만 원의 약식명령을 받았다.  피고인은 위와 같이 음주운전 금지규정을 위반한 사람으로서 다시 2021. 1. 24. 04:45경 혈중알코올농도 0.103%의 술에 취한 상태로 안산시 상록구 구룡로 87에 있는 근로복지공단 안산병원 앞 도로에서부터 같은 구 호동로 118에 있는 일동파출소 앞도로까지 약 1. 3km 구간에서 B 포터2 화물차를 운전하였다.  이로써 피고인은 도로교통법 제44조 제1항을 2회 위반하였다.  증거의 요지  1. 피고인의 법정진술  1. 수사보고(주취운전자 정황보고)의 기재  1. 교통사고 발생상황보고, 교통사고 발생보고서, 음주측정 기록지, 음주운전단속결과통보, 주취운전자 정황진술보고서, 교통사고실황조사서, 관련사진, 차적조회의 각 기재 및 영상  1. 판시 전과 : 범죄경력등조회회보서, 수사보고(음주운전 전력 확인 및 약식명령 첨부)의 각 기재  법령의 적용  1. 범죄사실에 대한 해당법조 및 형의 선택  도로교통법 제148조의2 제1항, 제44조 제1항, 벌금형 선택  1. 노역장유치  형법 제70조 제1항, 제69조 제2항  1. 가납명령  형사소송법 제334조 제1항  양형의 이유  현행 도로교통법은 도로교통 안전을 위협하는 음주운전을 미연에 방지하고, 이에 대한 경각심을 일깨울 목적으로 음주운전 또는 음주측정거부 금지조항을 2회 이상 위반한 사람을 엄벌하도록 규정하고 있는 점, 피고인이 음주운전으로 벌금형으로 1차례 처벌받은 전력이 있음에도 다시 음주운전을 한 점, 음주운전은 자신의 생명뿐만 아니라 타인의 생명까지 해할 수 있는 위험한 범행인 점, 혈중알코올농도의 수치가 적지 않은 점 등은 불리한 정상이다.  다만 피고인이 자신의 잘못을 반성하고 있는 점 등을 유리한 정상으로 참작하고, 그 밖에 피고인 나이, 성행, 환경, 범행동기, 수단과 결과, 범행 후 정황 등 제반 양형조건을 종합하여, 주문과 같이 형을 정한다.")))

1/1 [==============================] - 0s 315ms/step
[[0.]]
1/1 [==============================] - 0s 43ms/step
[[0.]]
1/1 [==============================] - 0s 48ms/step
[[11.397578]]
1/1 [==============================] - 0s 42ms/step
[[0.]]
1/1 [==============================] - 0s 34ms/step
[[0.]]
1/1 [==============================] - 0s 32ms/step
[[0.]]
1/1 [==============================] - 0s 52ms/step
[[0.]]
1/1 [==============================] - 0s 40ms/step
[[0.]]
1/1 [==============================] - 0s 30ms/step
[[688.72955]]
